# AutoML: Ray Tune example

First, install the dependencies using `pip` -- which should work well even if you're using `conda` to manage Python packages.
If you've these installed already the commands will do nothing.
In any case, this is needed for using many of the cloud-based services for running notebooks.

In [1]:
!pip install ray[rllib]
!pip install ray[debug]
!pip install ray[tune]
!pip install bayesian-optimization

In [2]:
import ray
import ray.tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

In [3]:
import time

def easy_objective (config, reporter):
    time.sleep(0.2)
    
    for i in range(config["iterations"]):
        reporter(
            timesteps_total=i,
            mean_loss=(config["height"] - 14)**2 - abs(config["width"] - 3)
            )

        time.sleep(0.02)

In [4]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test",
    action="store_true",
    help="Finish quickly for testing"
)

args, _ = parser.parse_known_args()

In [5]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2020-05-08 10:23:20,191	INFO resource_spec.py:212 -- Starting Ray with 3.71 GiB memory available for workers and up to 1.87 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-08 10:23:20,663	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:11888',
 'object_store_address': '/tmp/ray/session_2020-05-08_10-23-20_173564_28842/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-08_10-23-20_173564_28842/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-05-08_10-23-20_173564_28842'}

In [6]:
space = { "width": (0, 20), "height": (-100, 100) }

algo = BayesOptSearch(
    space,
    metric="mean_loss",
    mode="min",
    utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
    }
)

In [7]:
config = {
    "num_samples": 10 if args.smoke_test else 1000,
    "config": {
        "iterations": 10
    },
    "stop": {
        "timesteps_total": 100
    }
}

In [8]:
scheduler = AsyncHyperBandScheduler(metric="mean_loss", mode="min")

In [ ]:
ray.tune.run(
    easy_objective,
    name="my_exp",
    search_alg=algo,
    scheduler=scheduler,
    **config
)

Trial name,status,loc,height,width
easy_objective_bb775770,RUNNING,,-16.5956,14.4065
easy_objective_bb785206,PENDING,,-99.9771,6.04665
easy_objective_bb78a38c,PENDING,,-70.6488,1.84677
easy_objective_bb78f1c0,PENDING,,-62.748,6.91121
easy_objective_bb79232a,PENDING,,-20.6465,10.7763
easy_objective_bb794dc8,PENDING,,-16.1611,13.7044
easy_objective_bb79a962,PENDING,,-59.1096,17.5623
easy_objective_bb79fcc8,PENDING,,-94.5225,13.4094
easy_objective_bb7a90ca,PENDING,,-16.539,11.1738
easy_objective_bb7ad5a8,PENDING,,-71.9226,3.96203


(pid=28880) 2020-05-08 10:24:13,538	INFO trainable.py:217 -- Getting current IP.
(pid=28878) 2020-05-08 10:24:13,538	INFO trainable.py:217 -- Getting current IP.
(pid=28879) 2020-05-08 10:24:13,544	INFO trainable.py:217 -- Getting current IP.
(pid=28881) 2020-05-08 10:24:13,555	INFO trainable.py:217 -- Getting current IP.
Result for easy_objective_bb78a38c:
  date: 2020-05-08_10-24-13
  done: false
  experiment_id: 9dcf00ceaf3b4692a6e4f45135be6f70
  experiment_tag: 3_height=-70.649,iterations=10,width=1.8468
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: 7164.269810215997
  neg_mean_loss: -7164.269810215997
  node_ip: 192.168.1.65
  pid: 28878
  time_since_restore: 0.20344305038452148
  time_this_iter_s: 0.20344305038452148
  time_total_s: 0.20344305038452148
  timestamp: 1588958653
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: bb78a38c
  
Result for easy_objective_bb78f1c0:
  date: 2020-05-08_10-24-13
  done: false
  experiment_id

Trial name,status,loc,height,width,loss,iter,total time (s),ts
easy_objective_bb775770,TERMINATED,,-16.5956,14.4065,924.684,10,1.09079,9
easy_objective_bb785206,TERMINATED,,-99.9771,6.04665,12987.7,1,0.204294,0
easy_objective_bb78a38c,TERMINATED,,-70.6488,1.84677,7164.27,4,0.35847,3
easy_objective_bb78f1c0,TERMINATED,,-62.748,6.91121,5886.34,4,0.353056,3
easy_objective_bb79232a,RUNNING,192.168.1.65:28891,-20.6465,10.7763,1192.6,1,0.204764,0
easy_objective_bb794dc8,RUNNING,,-16.1611,13.7044,,,,
easy_objective_bb79a962,RUNNING,,-59.1096,17.5623,,,,
easy_objective_bb79fcc8,RUNNING,,-94.5225,13.4094,,,,
easy_objective_bb7a90ca,PENDING,,-16.539,11.1738,,,,
easy_objective_bb7ad5a8,PENDING,,-71.9226,3.96203,,,,


(pid=28889) 2020-05-08 10:24:17,587	INFO trainable.py:217 -- Getting current IP.
Result for easy_objective_bb794dc8:
  date: 2020-05-08_10-24-17
  done: false
  experiment_id: e7561add7c5b4f859f1d04d6033d1fdf
  experiment_tag: 6_height=-16.161,iterations=10,width=13.704
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: 898.987389434387
  neg_mean_loss: -898.987389434387
  node_ip: 192.168.1.65
  pid: 28890
  time_since_restore: 0.20436716079711914
  time_this_iter_s: 0.20436716079711914
  time_total_s: 0.20436716079711914
  timestamp: 1588958657
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: bb794dc8
  
Result for easy_objective_bb79a962:
  date: 2020-05-08_10-24-17
  done: true
  experiment_id: af8e4d312a8c4399a2815231fac7a8af
  experiment_tag: 7_height=-59.11,iterations=10,width=17.562
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: 5330.443960326138
  neg_mean_loss: -5330.443960326138
  node_ip: 192.168.1.65
  pid: 288

Trial name,status,loc,height,width,loss,iter,total time (s),ts
easy_objective_bf5f513a,PENDING,,-17.3095,14.0182,,,,
easy_objective_bf970ff8,PENDING,,-15.8852,12.8881,,,,
easy_objective_bfd98c8e,PENDING,,-16.0086,12.8853,,,,
easy_objective_c07c2548,PENDING,,-16.3361,12.8755,,,,
easy_objective_c0cb6c7a,PENDING,,-16.0536,12.7919,,,,
easy_objective_c1171206,PENDING,,-16.0649,12.7919,,,,
easy_objective_bd2d5038,RUNNING,,-17.0702,13.9803,,,,
easy_objective_bd51551e,RUNNING,,-16.5956,14.4065,,,,
easy_objective_bf2690fc,RUNNING,,-16.8992,14.2493,,,,
easy_objective_bb775770,TERMINATED,,-16.5956,14.4065,924.684,10,1.09079,9


...

Result for easy_objective_766d9512:
  date: 2020-05-08_10-29-43
  done: true
  experiment_id: cf48b78a90d94a72871cc856ae536f41
  experiment_tag: 208_height=14.399,iterations=10,width=19.607
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.447341490677335
  neg_mean_loss: 16.447341490677335
  node_ip: 192.168.1.65
  pid: 29133
  time_since_restore: 0.20380806922912598
  time_this_iter_s: 0.20380806922912598
  time_total_s: 0.20380806922912598
  timestamp: 1588958983
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: 766d9512
  
(pid=29135) 2020-05-08 10:29:45,847	INFO trainable.py:217 -- Getting current IP.
Result for easy_objective_74173f52:
  date: 2020-05-08_10-29-38
  done: true
  experiment_id: 76fb4120cc504cd2a553d507aa37d8d3
  experiment_tag: 206_height=14.601,iterations=10,width=19.552
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.190165235276883
  neg_mean_loss: 16.190165235276883
  node_ip: 192.168.1.65
  

Trial name,status,loc,height,width,loss,iter,total time (s),ts
easy_objective_7e476650,PENDING,,13.946,19.5706,,,,
easy_objective_7f2bf036,PENDING,,13.8346,19.4712,,,,
easy_objective_810a7a62,PENDING,,14.1567,19.7333,,,,
easy_objective_821c4840,PENDING,,14.0124,19.847,,,,
easy_objective_8318979e,PENDING,,14.2529,19.9743,,,,
easy_objective_84a43460,PENDING,,13.8485,19.9514,,,,
easy_objective_791e21dc,RUNNING,192.168.1.65:29135,14.2582,19.9509,-16.8843,10,3.70743,9
easy_objective_7a92c18a,RUNNING,,13.9836,19.9391,,,,
easy_objective_7b981486,RUNNING,,13.9739,19.8726,,,,
easy_objective_7d2fe72e,RUNNING,,14.2459,19.9753,,,,


(pid=29137) 2020-05-08 10:29:52,116	INFO trainable.py:217 -- Getting current IP.
Result for easy_objective_7d2fe72e:
  date: 2020-05-08_10-29-52
  done: false
  experiment_id: b7c529602ab24aaa9b4c3da453a55795
  experiment_tag: 213_height=14.246,iterations=10,width=19.975
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.91483030629931
  neg_mean_loss: 16.91483030629931
  node_ip: 192.168.1.65
  pid: 29137
  time_since_restore: 0.20363092422485352
  time_this_iter_s: 0.20363092422485352
  time_total_s: 0.20363092422485352
  timestamp: 1588958992
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: 7d2fe72e
  
Result for easy_objective_7b981486:
  date: 2020-05-08_10-29-50
  done: false
  experiment_id: 233dea613eea4aafa7b65c02e1a0050c
  experiment_tag: 212_height=13.974,iterations=10,width=19.873
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.871959248379287
  neg_mean_loss: 16.871959248379287
  node_ip: 192.168.1.65
  

Trial name,status,loc,height,width,loss,iter,total time (s),ts
easy_objective_821c4840,PENDING,,14.0124,19.847,,,,
easy_objective_8318979e,PENDING,,14.2529,19.9743,,,,
easy_objective_84a43460,PENDING,,13.8485,19.9514,,,,
easy_objective_85bd3004,PENDING,,14.8102,19.793,,,,
easy_objective_86d7885e,PENDING,,13.6499,19.9188,,,,
easy_objective_88666258,PENDING,,14.406,19.9938,,,,
easy_objective_7a92c18a,RUNNING,192.168.1.65:29138,13.9836,19.9391,-16.9389,10,3.22304,9
easy_objective_7e476650,RUNNING,,13.946,19.5706,,,,
easy_objective_7f2bf036,RUNNING,,13.8346,19.4712,,,,
easy_objective_810a7a62,RUNNING,,14.1567,19.7333,,,,


(pid=29142) 2020-05-08 10:29:59,006	INFO trainable.py:217 -- Getting current IP.
Result for easy_objective_810a7a62:
  date: 2020-05-08_10-29-59
  done: false
  experiment_id: d4dd6cc006434770945224f3c0c53c6e
  experiment_tag: 216_height=14.157,iterations=10,width=19.733
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.708757016280014
  neg_mean_loss: 16.708757016280014
  node_ip: 192.168.1.65
  pid: 29142
  time_since_restore: 0.20360779762268066
  time_this_iter_s: 0.20360779762268066
  time_total_s: 0.20360779762268066
  timestamp: 1588958999
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: 810a7a62
  
Result for easy_objective_7f2bf036:
  date: 2020-05-08_10-29-56
  done: true
  experiment_id: 08879953a2354deaa011e91f538be06a
  experiment_tag: 215_height=13.835,iterations=10,width=19.471
  hostname: derwen
  iterations_since_restore: 1
  mean_loss: -16.44382535511015
  neg_mean_loss: 16.44382535511015
  node_ip: 192.168.1.65
  p